In [ ]:
import numpy as np
import pandas as pd
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils
from string import punctuation

In [ ]:
features = pd.read_csv("final_df2.csv")
features.head(20)

,Unnamed: 0,movie,release date,rating,genre,language,directors,duration,Box Office,Description,Audience Score,Tomatometer,Reviews,compound_audience,neutral_audience,positive_audience,negative_audience,positive,negative,neutral,compound,aud_asp_neutral,aud_asp_negative,aud_asp_positive,critic_reviews
0,0,chicken_run,"Jun 21, 2000",G,"['animation', 'kids and family', 'comedy', 'ad...",English,['Peter Lord'],1h 25m,NaN,"This engaging stop-motion, claymation adventur...",65.0,97.0,['While Chicken Run has a great animation styl...,0.395363,0.764375,0.208500,0.027125,0.140000,0.067500,0.792500,0.130605,0.127229,0.261164,0.611607,"Chicken Run, if anything, offers an alternativ..."
1,1,the_taste_of_others,"Feb 26, 2002",R,['comedy'],French (France),['Agnès Jaoui'],1h 52m,$635.3K,Castella (Jean-Pierre Bacri) is a successful b...,82.0,100.0,['A nice comedy that has pokes some friendly b...,0.544600,0.700125,0.237250,0.062625,0.255050,0.030100,0.714900,0.400550,0.044488,0.232734,0.722778,Succeeds with believable characters and situat...
2,2,yi_yi,"May 8, 2001",NaN,['drama'],Edward Yang,['Edward Yang'],2h 53m,$969.1K,"Set in Taiwan, the film follows the lives of t...",91.0,97.0,['A city symphony in minor key with a city-siz...,0.559800,0.806111,0.152556,0.041444,0.206000,0.040300,0.753700,0.348930,0.169338,0.233988,0.596674,"I felt like I had lived it, and not just becau..."
3,3,memento,"Mar 1, 2017",R (Language|Drug Content|Violence),['mystery and thriller'],English,['Christopher Nolan'],1h 56m,$25.5M,Leonard (Guy Pearce) is tracking down the man ...,94.0,93.0,"['Not a masterpiece but really good', ""Memento...",0.326714,0.740143,0.205429,0.054571,0.108600,0.056200,0.835150,0.157130,0.204062,0.217740,0.578198,Think of all the ways that a linear film struc...
4,4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",80.0,97.0,"[""Dedicated to the millions of Detroit Tigers ...",0.840233,0.697333,0.289333,0.013333,0.223750,0.024300,0.751950,0.574785,0.047392,0.034910,0.917698,It's an adroit manipulation of interviews and ...
5,5,you_can_count_on_me,"Nov 10, 2000",R (Scene of Sexuality|Language|Some Drug Use),['drama'],English,['Kenneth Lonergan'],1h 49m,$9.2M,Sammy is a single mother who is extremely prot...,88.0,95.0,"[""It's hard to criticize this movie as anythin...",0.478225,0.673750,0.114500,0.212250,0.238200,0.036650,0.725250,0.397865,0.216037,0.668246,0.115717,Unusually sharp editing is the first hint that...
6,6,1103281-traffic,"Jan 5, 2001",R (Some Sexuality|Pervasive Drug Content|Stron...,"['drama', 'crime']",English,['Steven Soderbergh'],2h 27m,$124.1M,A contemporary thriller set in the world of dr...,85.0,92.0,"[""Saw this 21 years ago when it first came out...",0.253400,0.637250,0.174750,0.187750,0.150200,0.116050,0.733850,0.069335,0.101956,0.464714,0.433330,"Tons of drug use, violence, and depressing sto..."
7,7,best_in_show,"Sep 29, 2000",PG-13,"['comedy', 'gay and lesbian']",English,['Christopher Guest'],1h 30m,$18.6M,"The tension is palpable, the excitement is mou...",89.0,94.0,"[""Christopher Guest's best film. Almost every ...",0.769160,0.756800,0.226000,0.017200,0.146200,0.011900,0.841900,0.310145,0.094226,0.210409,0.695364,Guest seems to know just how far he can stretc...
8,8,croupier,"Apr 21, 1998",NaN,['mystery and thriller'],Mike Hodges,['Mike Hodges'],1h 31m,$6.2M,Jack Manfred (Clive Owen) is an aspiring write...,79.0,96.0,"[""A film comes along occasionally that simply ...",0.374360,0.748600,0.163600,0.087800,0.207333,0.046722,0.746111,0.273650,0.058734,0.505459,0.435807,Script and direction play safe throughout.Cool...
9,9,not_one_less,"Jan 27, 2015",G,['drama'],Chinese,['Yimou Zhang'],1h 46m,$589.1K,"In a poverty-stricken village, a teacher goes ...",90.0,96.0,"['Simple, yet moving. A great insight into rur...",0.300414,0.739000,0.175571,0.085429,0.157357,0

In [ ]:
x = features['genre'].iloc[0]
features=features.dropna(subset=["genre"])

In [ ]:
def helper(x):
  l = re.findall('\[(.*)\]', x)[0].split(",")
  l2 = []
  for i in l:
    l2.append(i.strip().strip("'"))
  return l2

In [ ]:
pd.Series(features['genre'].apply(helper).sum()).value_counts(ascending = False)

drama                   1158
comedy                   470
documentary              462
mystery & thriller       233
adventure                209
action                   167
romance                  154
fantasy                  143
biography                125
crime                    122
history                  117
animation                 95
horror                    85
kids & family             83
sci-fi                    76
gay & lesbian             64
war                       55
music                     52
mystery and thriller      41
musical                   26
kids and family           23
western                   21
anime                     13
sci fi                    11
gay and lesbian           10
sports & fitness           7
other                      6
sports and fitness         3
stand-up                   1
dtype: int64

In [ ]:
genre="documentary"
selected_genre=features[features["genre"].str.contains(genre)]
selected_genre.head()

,Unnamed: 0,movie,release date,rating,genre,language,directors,duration,Box Office,Description,Audience Score,Tomatometer,Reviews,compound_audience,neutral_audience,positive_audience,negative_audience,positive,negative,neutral,compound,aud_asp_neutral,aud_asp_negative,aud_asp_positive,critic_reviews
4,4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",80.0,97.0,"[""Dedicated to the millions of Detroit Tigers ...",0.840233,0.697333,0.289333,0.013333,0.223750,0.02430,0.751950,0.574785,0.047392,0.034910,0.917698,It's an adroit manipulation of interviews and ...
11,11,dark_days,"Dec 4, 2015",NaN,['documentary'],Marc Singer,['Marc Singer'],1h 34m,$436.8K,"""Dark Days"" is a feature length documentary ab...",89.0,94.0,['Equal parts inspiring and depressing. A lite...,0.906725,0.703750,0.221500,0.075000,0.090550,0.10805,0.801450,-0.006655,0.159434,0.424432,0.416135,This is the world discovered and illuminated b...
12,12,filth_and_the_fury,"Mar 29, 2000",R (Strong Language|Sexual Content|Drug Use),['documentary'],English,['Julien Temple'],1h 48m,$606.6K,"Assembled from unseen archive footage, rare of...",88.0,95.0,"[""Nevermind the Bollocks Here's the Sex Pistol...",-0.026756,0.745000,0.143222,0.111778,0.034091,0.00000,0.965909,0.057900,0.130965,0.411622,0.457413,"[]Best for older teens, especially Sex Pistols..."
20,20,wattstax,"Feb 18, 2016",R,"['music', 'documentary']",English,['Mel Stuart'],1h 38m,NaN,"In 1972 at the Los Angeles Coliseum, director ...",90.0,90.0,"['Black people of america, I Love You', 'Conju...",0.666143,0.800000,0.181714,0.018286,0.172250,0.03780,0.789950,0.423465,0.097406,0.077102,0.825492,All [the performers] draw lively reactions fro...
35,35,original_kings_of_comedy,"Aug 18, 2000",R (Language and Sex Related Humor),"['comedy', 'documentary']",English,['Spike Lee'],1h 55m,$38.2M,A cultural phenomenon for our time: Spike Lee ...,79.0,83.0,"[""Though an appealing lineup, the capturing do...",0.606740,0.649400,0.300200,0.050400,0.144250,0.05435,0.801400,0.317675,0.155835,0.421047,0.423119,[Spike Lee] knows the show belongs to his come...


In [ ]:
combined = ' '.join(selected_genre["critic_reviews"])
combined

'It\'s an adroit manipulation of interviews and stock footage, mostly along the lines of establishing not so much what Hammerin\' Hank did for himself, but what he meant to American Jews.Kempner demonstrates how the star\'s success and dignified bearing inspired a generation of Jews to fight through the ethnic barriers in all fields.Writer-director Aviva Kempner crafted this love letter to Greenberg by imaginatively weaving interviews with great footage from newsreels and feature films.Aviva Kempner hits a grand slam in her thrilling documentary about baseball, about a great baseball player, about Jews in America and about the country itself in the 1930s and 1940s.The Life and Times of Hank Greenberg is a gem of a baseball documentary because it illuminates what the Hall of Famer\'s splendid career meant away from the ballpark.Aviva Kempner\'s warm and intelligent mash note to a man who clearly deserved it.Aviva Kempner\'s documentary The Life and Times of Hank Greenberg does what a hi

In [ ]:
combined_final=combined.lower()
combined_final = combined_final.translate(str.maketrans("", "", punctuation))

In [ ]:
characters = sorted(list(set(combined_final)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [ ]:
X = []
Y = []
length = len(combined_final)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = combined_final[i:i + seq_length]
    label =combined_final[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [ ]:
model = Sequential([
    LSTM(300, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(300, return_sequences=True),
    Dropout(0.2),
    LSTM(300),
    Dropout(0.2),
    Dense(Y_modified.shape[1], activation="softmax"),
])

path = f"output/{genre}.h5"
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_modified, Y_modified, epochs=50, batch_size=100)

model.save_weights(path)

Epoch 1/50
    8/11197 [..............................] - ETA: 9:09:45 - loss: 3.4099

KeyboardInterrupt: ignored

In [ ]:
model.load_weights(f"output/{genre}.h5")

In [ ]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
for i in range(1000):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [ ]:
text=""
for char in full_string:
    text = text+char
text